In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
import configparser 
import datetime

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata

In [2]:
#Берем данные по БД

config = configparser.ConfigParser()
config.read('config.ini')
conn_string = config.get('DATABASE', 'connection_url')

In [11]:
#Загружаем данные в БД


#Загрузка модели для генерации случайных данных

filepath='my_synthesizer.pkl'
synthesizer = GaussianCopulaSynthesizer.load(filepath)

synthetic_data = synthesizer.sample(num_rows=10)

#Убираем баг и подставляем дату

synthetic_data['dt'] = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')

#Баг с обучением

synthetic_data['3SsnPorch'] = 0

#Функция чтения и записи в БД

def upload_to_db(df_in,db_table_out):
    
    try:

        db = create_engine(conn_string) 
        with db.connect() as conn:
            
            df_in.to_sql(db_table_out, con=conn, if_exists='append',index=False) 
            print('Uploaded')
    finally:
        print('OK')

#Добавить логирование в файл

In [12]:
df_in = synthetic_data
db_table_out = 'house_prices_generator'

upload_to_db(df_in,db_table_out)

Uploaded
OK
